In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import pandas as pd 

from scipy import optimize
from scipy import integrate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns


sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)




In [2]:
#we pivot the columns in order to make it for rolling the functions 
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.reset_index().pivot(columns='country', index='date', values='confirmed')
pd_JH_data.reset_index(drop=True,inplace=True)
df_analyse = pd_JH_data.rename_axis(None, axis=1)
df_analyse.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,2,0,0,0,0,0


In [3]:
N0=1000000

In [4]:
def SIR_model(SIR,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return([dS_dt,dI_dt,dR_dt])


# Simulative approach to calculate SIR curves

In [5]:
# set some basic parameters
# beta/gamma is denoted as  'basic reproduction number'
def simulate_SIR(country='Germany'):
     #max susceptible population
    beta=0.4   # infection spread dynamics
    gamma=0.1  # recovery rate


    # condition I0+S0+R0=N0
    I0=df_analyse[country][35]
    S0=N0-I0
    R0=0
    SIR=np.array([S0,I0,R0])
    propagation_rates_sumulation=pd.DataFrame(columns={'susceptible':S0,
                                            'infected':I0,
                                            'recoverd':R0})



    for each_t in np.arange(100):

        new_delta_vec=SIR_model(SIR,beta,gamma)

        SIR=SIR+new_delta_vec

        propagation_rates_sumulation=propagation_rates_sumulation.append({'susceptible':SIR[0],
                                                    'infected':SIR[1],
                                                    'recovered':SIR[2]}, ignore_index=True)


    return list(propagation_rates_sumulation.index),propagation_rates_sumulation.infected.tolist()

In [6]:
#simulate_SIR()

In [7]:
#simulate_SIR('Italy')

# Fitting the parameters of SIR model

In [8]:
# ensure re-initialization 
def fitting_SIR_model(country='Germany'):
    
    def SIR_model_t(SIR,t,beta,gamma):
        ''' Simple SIR model
            S: susceptible population
            t: time step, mandatory for integral.odeint
            I: infected people
            R: recovered people
            beta: 

            overall condition is that the sum of changes (differnces) sum up to 0
            dS+dI+dR=0
            S+I+R= N (constant size of population)

        '''

        S,I,R=SIR
        dS_dt=-beta*S*I/N0          #S*I is the 
        dI_dt=beta*S*I/N0-gamma*I
        dR_dt=gamma*I
        return dS_dt,dI_dt,dR_dt

    def fit_odeint(x, beta, gamma):
        '''
        helper function for the integration
        '''
        return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

    ydata = np.array(df_analyse[country][35:])
    t=np.arange(len(ydata))
    I0=ydata[0]
    S0=N0-I0
    R0=0
    beta=0.4 

    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
    perr = np.sqrt(np.diag(pcov))

    print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
    print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])


    fitted=fit_odeint(t, *popt)
    return list(t),list(ydata),list(fitted)

In [9]:
#fitting_SIR_model('Italy')

In [10]:
#fitting_SIR_model()

# Dynamic beta in SIR (infection rate)

In [11]:
def dynamic_beta_SIR(country='Germany'):
    t_initial=28
    t_intro_measures=14
    t_hold=21
    t_relax=21
    
    end_col=35+t_initial+t_intro_measures+t_hold+t_relax

    ydata = np.array(df_analyse[country][35:end_col])
    I0=ydata[0]
    #N0=1000000
    S0=N0-I0
    R0=0

    beta_max=0.4
    beta_min=0.11
    gamma=0.1
    pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                           np.linspace(beta_max,beta_min,t_intro_measures),
                           np.array(t_hold*[beta_min]),
                            np.linspace(beta_min,beta_max,t_relax),
                           ))

    SIR=np.array([S0,I0,R0])
    propagation_rates_dynamic=pd.DataFrame(columns={'susceptible':S0,
                                            'infected':I0,
                                            'recoverd':R0})



    for each_beta in pd_beta:

        new_delta_vec=SIR_model(SIR,each_beta,gamma)

        SIR=SIR+new_delta_vec

        propagation_rates_dynamic=propagation_rates_dynamic.append({'susceptible':SIR[0],
                                                    'infected':SIR[1],
                                                    'recovered':SIR[2]}, ignore_index=True)
    return list(propagation_rates_dynamic.index),list(ydata),propagation_rates_dynamic.infected.to_list()


In [12]:
#dynamic_beta_SIR()

In [13]:
#sample code for futher logic
import pandas as pd
 
df = pd.DataFrame({'key1': [], 'key2': [], 'key3': []})
for i in range(10):
    df = df.append({'key1': 'inddia'+str(i), 'key2': [1,2], 'key3': [2.3]}, ignore_index=True)
    
row=df.loc[df['key1'] == 'inddia0']
row.iloc[0]['key3']

[2.3]

In [14]:
list_of_countries=list(df_analyse.columns)

In [15]:
df_dynamic_SIR = pd.DataFrame({'country': [],'x_axis':[] ,'actual': [], 'predicted': []})
for country in list_of_countries:
    try:
        x_axis,actual,predicted=dynamic_beta_SIR(country)
    except:
        print("Error for country:",country)
        continue
    df_dynamic_SIR=df_dynamic_SIR.append({'country': country,'x_axis':x_axis ,'actual': actual, 
                                          'predicted': predicted},ignore_index=True)
    

In [16]:
df_dynamic_SIR.head()

,country,x_axis,actual,predicted
0,Afghanistan,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 5, 7, ...","[1.2999996, 1.689998752000432, 2.1969970796823..."
1,Albania,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 10, 12...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Algeria,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 1, 1, 1, 1, 3, 5, 12, 12, 17, 17, 19, 20, ...","[1.2999996, 1.689998752000432, 2.1969970796823..."
3,Andorra,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Angola,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [17]:
final_country_list=[]
df_fitting_SIR = pd.DataFrame({'country': [],'x_axis':[] ,'actual': [], 'predicted': []})
for country in list_of_countries:
    try:
        x_axis,actual,predicted=fitting_SIR_model(country)
    except:
        print("Error for country:",country)
        continue
    final_country_list.append(country)
    df_fitting_SIR=df_fitting_SIR.append({'country': country,'x_axis':x_axis ,'actual': actual, 
                                          'predicted': predicted},ignore_index=True)

/home/vikas/anaconda3/envs/ds/lib/python3.6/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/vikas/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in double_scalars
/home/vikas/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in double_scalars
/home/vikas/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


standard deviation errors :  [0.00388644 0.00390821]  start infect: 1
Optimal parameters: beta = 0.2554533014557034  and gamma =  0.17906954100208414
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0


/home/vikas/anaconda3/envs/ds/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/vikas/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in double_scalars


Error for country: Algeria
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [0.00325678 0.00303635]  start infect: 15
Optimal parameters: beta = 0.22048352444296387  and gamma =  0.17163583027828117
standard deviation errors :  [0.00704673 0.00689974]  start infect: 2
Optimal parameters: beta = 0.27141334525744737  and gamma =  0.20681915099253106
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [inf inf]  

Error for country: India
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [0.00099056 0.0009231 ]  start infect: 139
Optimal parameters: beta = 0.08328380597928788  and gamma =  0.021832014511949892
standard deviation errors :  [14.38974787 14.1235391 ]  start infect: 5
Optimal parameters: beta = 1.0118858719425488  and gamma =  0.9666349639314826
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [0.00165223 0.00148757]  start infect: 2
Optimal parameters: beta = 0.14589971796154805  and gamma =  0.07789508936704648
standard deviation errors :  [0.0015642  0.00113072]  start infect: 453
Optimal parameters: beta = 0.09413628091989112  and gamma =  0.02783915555271406
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [0.00227165 0.00208864

standard deviation errors :  [0.00214223 0.00159333]  start infect: 13
Optimal parameters: beta = 0.09240843519382182  and gamma =  0.01957439344475207
standard deviation errors :  [0.01008354 0.01001635]  start infect: 1
Optimal parameters: beta = 0.6886606760115564  and gamma =  0.6305893091202821
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [0.00281047 0.00281096]  start infect: 2
Optimal parameters: beta = 0.18270964762755354  and gamma =  0.10501602293697977
standard deviation errors :  [0.00692474 0.00690889]  start infect: 1
Optimal parameters: beta = 0.24617084919531915  and gamma =  0.17210960533364988
standard deviation errors :  [inf inf]  start infect: 0
Optimal parameters: beta = 1.0  and gamma =  1.0
standard deviation errors :  [0.01629732 0.01591891]  start infect: 32
Optimal par

In [ ]:
# %load src/visualization/visualize.py
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Dashboard for SIR visualization

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in final_country_list],
        value=['US', 'Germany','Italy'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='approaches',
    options=[
        {'label': 'Fitting the curve approach ', 'value': 'fitting'},
        {'label': 'Dynamic beta', 'value': 'dynamic'},
    ],
    value='fitting',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('approaches', 'value')])
def update_figure(country_list,approach):
    
    #my_yaxis="Number of infected people out of 1M population"
    
    my_yaxis={'type':"log",'title':'Number of infected people out of 1M population'}
    
    if approach=='fitting':
        df_plot=df_fitting_SIR
    else:
        df_plot=df_dynamic_SIR





    traces = []
    for each in country_list:
        
        df=df_plot.loc[df_plot['country']==each]
        x_axis=df.iloc[0]['x_axis']
        y_actual=df.iloc[0]['actual']
        y_predicted=df.iloc[0]['predicted']

 


        traces.append(dict(x=x_axis,
                                y=y_actual,
                                mode='markers+lines',
                                opacity=0.9,
                                name=each+' Actual'
                        )
                )
        traces.append(dict(x=x_axis,
                            y=y_predicted,
                            mode='markers+lines',
                            opacity=0.9,
                            name=each+' Predicted'
                    )
            )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)


/home/vikas/Desktop/Data_science/ads_covis_19/notebooks
Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
